## REPASO 2

JULIA HERNÁNDEZ CÁRDENAS

In [22]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.neural_network import MLPClassifier
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.gaussian_process.kernels import RBF
from sklearn.gaussian_process import GaussianProcessRegressor
from scipy.stats import t 
from sklearn.svm import SVC
from sklearn.linear_model import LinearRegression, Ridge, Lasso, LogisticRegression
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.model_selection import KFold, train_test_split, cross_validate

In [23]:
df = pd.read_csv("brain_tumor_dataset.csv")
df.dropna()
df.drop_duplicates()
df = df.drop(columns=['Patient_ID'])
df = df.head(5000)

In [24]:
x = df.drop(columns=["Tumor_Type"])
y = df[["Tumor_Type"]]

In [25]:
num_features = ["Age", "Tumor_Size", "Survival_Rate", "Tumor_Growth_Rate"]
cat_features = ["Gender", 'Location', 'Histology', 'Stage', 
               'Radiation_Treatment', 'Surgery_Performed','Chemotherapy','Family_History','MRI_Result','Follow_Up_Required']
target = ["Tumor_Type"]
X = df[num_features + cat_features]
y = df[target].values.ravel()

In [26]:
X=pd.get_dummies(X, columns=['Gender','Location','Histology','Stage',
                            'Radiation_Treatment','Surgery_Performed','Chemotherapy','Family_History',
                            'MRI_Result','Follow_Up_Required'])
#X = X.astype(int)

In [27]:
scaler = StandardScaler().fit(X)
X = scaler.transform(X)

In [28]:
indice = list(range(X.shape[1]))
preprocessor = ColumnTransformer(
    transformers=[('num', StandardScaler(), indice)],
    remainder='drop'
)

kfolds = KFold(n_splits=10, shuffle=True, random_state=42)

scoring = {
    'accuracy': 'accuracy',
    'auc_ovr': 'roc_auc_ovr' 
}

resultados = []

C_values = [0.1, 0.2, 0.3, 0.4, 0.5]
kernels = ['linear'] 

for k in kernels:
    for C_val in C_values:
        
        pipeline = Pipeline(steps=[
            ('preprocessor', preprocessor),
            ('model', SVC(
                C=C_val,          
                kernel=k, 
                gamma='scale', 
                degree=2, 
                probability=True,   
                random_state=42
            ))
        ])
        
        scores = cross_validate(pipeline, X, y, cv=kfolds, scoring=scoring, n_jobs=-1)
        
        resultados.append({
            'modelo': f'svc_{k}',
            'C': C_val,
            'accuracy_mean': np.mean(scores['test_accuracy']),
            'accuracy_std':  np.std(scores['test_accuracy']),
            'AUC_ovr_mean': np.mean(scores['test_auc_ovr']),
        })

In [30]:
resultados_df = pd.DataFrame(resultados)
print(resultados_df.sort_values(by='AUC_ovr_mean', ascending=False))

       modelo    C  accuracy_mean  accuracy_std  AUC_ovr_mean
4  svc_linear  0.5         0.4916      0.008800      0.503185
0  svc_linear  0.1         0.4942      0.010713      0.503125
1  svc_linear  0.2         0.4928      0.008588      0.503051
2  svc_linear  0.3         0.4918      0.008829      0.502659
3  svc_linear  0.4         0.4908      0.009765      0.501863
